In [15]:
import ciropt as co
import cvxpy as cp

In [19]:
L_smooth = 1.
mu = 0.0001

Capacitance = 2.
Inductance = 2.
R = 1

solver = "ca"
# solver = "ca_canonical"
# solver = "ca_canonical_X"
# solver = "gp_canonical_X"

time_limit = 200

# Ciropt problem

In [27]:
from ciropt.circuit_algorithms import define_function
from ciropt.circuit_opt import CircuitOpt
from ciropt.function import SmoothStronglyConvexFunction, StronglyConvexFunction, \
                            SmoothConvexFunction, ConvexFunction

problem = CircuitOpt()
func_f = define_function(problem, mu, L_smooth) 
func_g = define_function(problem, mu, L_smooth) 
h, alpha, beta, b, d = problem.h, problem.alpha, problem.beta, problem.b, problem.d

x1_star, y1_star, g_star = func_g.stationary_point()
x2_star, y2_star, f_star = func_f.stationary_point()


x_2_1 = problem.set_initial_point()
y_2_1, f_2_1 = func_f.oracle(x_2_1)

x_1_1 = x_2_1 - R * y_2_1
v_C_1 = -x_1_1

# x_2_2 = func_g.proximal_step(x_1_1, R)
y_1_1, g_1_1 = func_g.oracle(x_1_1)
x_2_2 = x_2_1 - R * y_1_1


y_2_2, f_2_2 = func_f.oracle(x_2_2)
x_1_2 = x_2_2 - R * y_2_2
v_C_2 = -x_1_2


E_1 = (Capacitance/2) * (v_C_1 - x1_star)**2
E_2 = (Capacitance/2) * (v_C_2 - x1_star)**2
Delta_1 = b * (f_2_1 + g_1_1 - f_star - g_star) - (y1_star + y2_star) * ( x_1_1 + x_2_1 -  x1_star - x2_star)
problem.set_energy(E_1 = E_1, E_2 = E_2)
problem.set_delta( Delta_1 )
problem.obj = b #+ d

In [28]:
# problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)

res, model, sp_exp = problem.solve(solver=solver, verbose=False, time_limit=100)[:3]
print(res)

dim_G=6, dim_F=5
opts={'ipopt.max_iter': 50000, 'ipopt.print_level': 0, 'print_time': 0, 'ipopt.sb': 'yes'}
None


In [24]:
res

# PEP verification

In [5]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] 
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [6]:
problem = PEPit.PEP()
func = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu,   # Strong convexity param.
                    L=L_smooth) 


x_P_star = func.stationary_point()
g_star, f_star = func.oracle(x_P_star)
x_P_1 = problem.set_initial_point()

g_1, f_1 = func.oracle(x_P_1)
x_P_1p5 = x_P_1 - (alpha * h / Capacitance) * g_1 
g_1p5, f_1p5 = func.oracle(x_P_1p5)
x_P_2 = x_P_1  - (beta * h / Capacitance) * g_1 - ((1 - beta) * h / Capacitance) * g_1p5

g_2, f_2 = func.oracle(x_P_2)

# Set the performance metric to the function values accuracy
E_1 = (Capacitance/2) * (x_P_1 - x_P_star)**2
E_2 = (Capacitance/2) * (x_P_2 - x_P_star)**2
Delta_1 = b * (f_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.3067190662852113e-09
